# Analyze data using ilastik
The notebook shows how load image contained in a dataset
from OMERO and analyze them in ilastik.
It assumes that ilastik project is linked to the dataset.
The metadata are read from OMERO and the Binary data are read from S3.
The ilastik results are saved as a zarr file and saved back to OMERO.
The order might need to be adjusted depending on the ilastik project.

### Insert required packages

In [1]:
import numpy
import os
import itertools
import zarr
import dask.array as da
import s3fs

import tempfile

import omero.clients
from omero.gateway import BlitzGateway
from getpass import getpass
from collections import OrderedDict

import ilastik_main
from ilastik.applets.dataSelection.opDataSelection import PreloadedArrayDatasetInfo
import ipywidgets as widgets
# package for 3d visualization
from itkwidgets import view

### Create a connection to an OMERO server

In [2]:
HOST = 'wss://outreach.openmicroscopy.org/omero-ws'
conn = BlitzGateway(input("Username: "),
                    getpass("OMERO Password: "),
                    host=HOST, secure=True)
conn.connect()
conn.c.enableKeepAlive(60)

Username: trainer-1
OMERO Password: ········


True

### Enter the dataset ID

In [3]:
dataset_id = 6161

### Load the ilastik projects linked to the dataset

In [4]:
def load_model(dataset_id, path):
    dataset = conn.getObject("Dataset", dataset_id)
    # Go through all the annotations on the Dataset
    options = []
    for ann in dataset.listAnnotations():
        if isinstance(ann, omero.gateway.FileAnnotationWrapper):
            name = ann.getFile().getName()
            # Select the ilatisk project TODO: use namespace
            if name.endswith(".ilp"):
                file_path = os.path.join(path, name)
                options.append((name, file_path))
                with open(str(file_path), 'wb') as f:
                    for chunk in ann.getFileInChunks():
                        f.write(chunk)
    return widgets.Dropdown(options=options, disabled=False)

### Load an Image as 5D-numpy array: order TZYXC

In [5]:
def load_from_s3(image, resolution='0'):
    cache_size_mb = 2048
    # ID to be changed
    id = image.getId()
    cfg = {
        'anon': True,
        'client_kwargs': {
            'endpoint_url': 'https://s3.embassy.ebi.ac.uk',
        },
        'root': 'idr/outreach/%s.zarr' % id
    }
    s3 = s3fs.S3FileSystem(
        anon=cfg['anon'],
        client_kwargs=cfg['client_kwargs'],
    )
    store = s3fs.S3Map(root=cfg['root'], s3=s3, check=False)
    cached_store = zarr.LRUStoreCache(store, max_size=(cache_size_mb * 2**20))
    # data.shape is (t, c, z, y, x) by convention
    data = da.from_zarr(cached_store)
    values = data[:]
    values = values.swapaxes(1, 2).swapaxes(2, 3).swapaxes(3, 4)
    return numpy.asarray(values)

### Create a temporary directory where to download the ilastik project

In [6]:
path = tempfile.mkdtemp()
if not os.path.exists(path):
    os.makedirs(path)

### Select the ilastik project to use.

In [7]:
model_selection = load_model(dataset_id, path)
display(model_selection)

Dropdown(options=(('pixel-class-133.ilp', '/tmp/tmp1yv83uyd/pixel-class-133.ilp'),), value='/tmp/tmp1yv83uyd/p…

### Load each image as a 5D-numpy array and analyze.
Save the probabilities as Zarr zip and link it to the image.

In [8]:
# Load the model linked to the dataset
model_file = model_selection.value


images = conn.getObjects('Image', opts={'dataset': dataset_id})

# Prepare ilastik
os.environ["LAZYFLOW_THREADS"] = "2"
os.environ["LAZYFLOW_TOTAL_RAM_MB"] = "2000"
args = ilastik_main.parse_args([])
args.headless = True
args.project = model_file
shell = ilastik_main.main(args)

values = itertools.islice(images, 2)
for image in values:
    filename, file_extension = os.path.splitext(image.getName())
    input_data = load_from_s3(image)

    # run ilastik headless
    print('running ilastik using %s and %s' % (model_file, image.getName()))
    role_data_dict = OrderedDict(
    [
        (
            "Raw Data",
            [
                PreloadedArrayDatasetInfo(preloaded_array=input_data)
            ],
        )
    ])

    predictions = shell.workflow.batchProcessingApplet.run_export(role_data_dict, export_to_array=True)
    # Save the probabilities file to the image
    print("Saving Probabilities as zip attached to the Image in OMERO")
    namespace = "ilastik.zarr.demo"
    name = filename + "_Probabilities_zarr.zip"
    desc = "ilastik probabilities from Image:%s" % image.getId()
    for data in predictions:
        file_path = os.path.join(path, name)
        # Re-organise array from tzyxc to zctyx order expected by OMERO
        data = data.swapaxes(0, 1).swapaxes(3, 4).swapaxes(2, 3).swapaxes(1, 2)
        data_viewer = data[:, 0, 0, :, :]
print("done")

INFO ilastik_main: Starting ilastik from "/srv/conda/envs/notebook".


INFO:ilastik_main:Starting ilastik from "/srv/conda/envs/notebook".


Starting ilastik from "/srv/conda/envs/notebook".
INFO ilastik_main: Resetting lazyflow thread pool with 2 threads.


INFO:ilastik_main:Resetting lazyflow thread pool with 2 threads.


INFO ilastik_main: Configuring lazyflow RAM limit to 2.0GiB


...

DEBUG lazyflow.operators.classifierOperators: Features took 3.621525 seconds. Prediction took 1.1756630000000001 seconds. Subregion: start '[184, 184, 92, 0]' stop '[236, 275, 184, 2]'


DEBUG:lazyflow.operators.classifierOperators:Features took 3.621525 seconds. Prediction took 1.1756630000000001 seconds. Subregion: start '[184, 184, 92, 0]' stop '[236, 275, 184, 2]'


DEBUG lazyflow.operators.classifierOperators: Features took 2.130148 seconds. Prediction took 0.836557 seconds. Subregion: start '[184, 184, 184, 0]' stop '[236, 275, 271, 2]'


DEBUG:lazyflow.operators.classifierOperators:Features took 2.130148 seconds. Prediction took 0.836557 seconds. Subregion: start '[184, 184, 184, 0]' stop '[236, 275, 271, 2]'


Saving Probabilities as zip attached to the Image in OMERO
done


### View the results.

In [9]:
view(data_viewer)

Viewer(rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x7f0354361f…

### Close the connection to the OMERO server

In [10]:
conn.close()

### License
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.